# Medical insurance dataset Federated
https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification?hl=en
### Stand 09.11.
* Tensorflow Federated scheint zu funktionieren
    * Ergebnisse sehen deutlich schlechter aus als zentralisiert.
    * MAE geht nicht unter ~8700 (vs. ~2900 im zentralisierten Modell)
        * R² ist negativ!

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [1]:
%reload_ext tensorboard

In [1]:
import collections
import numpy as np
import pandas as pd
import tqdm
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import tensorflow_federated as tff
from tensorflow_addons.metrics import RSquare


np.random.seed(0)

2023-01-26 17:22:00.318918: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-26 17:22:00.318967: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-26 17:22:00.350010: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-26 17:22:21.140997: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-26 17:22:21.141242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [2]:
# Load data and preprocess it
dataset = pd.read_csv('data/insurance.csv')
# Create categorical data from non numerical features
dataset['sex'] = dataset['sex'].astype('category').cat.codes
dataset['region'] = dataset['region'].astype('category').cat.codes
dataset['smoker'] = dataset['smoker'].astype('category').cat.codes
# Min Max scale the features (which will not be one hot encoded)
scaler = MinMaxScaler()
scaler.fit(dataset[['age', 'bmi', 'children']])
dataset[['age', 'bmi', 'children']] = scaler.transform(dataset[['age', 'bmi', 'children']])
# Change region to an one hot encoding feature
dataset[['region0', 'region1', 'region2', 'region3']] = pd.get_dummies(dataset['region'])
dataset

,age,sex,bmi,children,smoker,region,charges,region0,region1,region2,region3
0,0.021739,0,0.321227,0.0,1,3,16884.92400,0,0,0,1
1,0.000000,1,0.479150,0.2,0,2,1725.55230,0,0,1,0
2,0.217391,1,0.458434,0.6,0,2,4449.46200,0,0,1,0
3,0.326087,1,0.181464,0.0,0,1,21984.47061,0,1,0,0
4,0.304348,1,0.347592,0.0,0,1,3866.85520,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1333,0.695652,1,0.403820,0.6,0,1,10600.54830,0,1,0,0
1334,0.000000,0,0.429379,0.0,0,0,2205.98080,1,0,0,0
1335,0.000000,0,0.562012,0.0,0,2,1629.83350,0,0,1,0
1336,0.065217,0,0.264730,0.0,0,3,2007.94500,0,0,0,1


## Data preprocessing and model initilisation

In [3]:
NUM_CLIENTS = 4
NUM_EPOCHS = 5
BATCH_SIZE = 10
SHUFFLE_BUFFER = 20
PREFETCH_BUFFER = 5
NUM_ROUNDS = 150
RUN_NAME = f'0,8-3({NUM_ROUNDS})-{NUM_EPOCHS}-epochs-{BATCH_SIZE}-batch-WithRegion/'

In [4]:
# Function to divide data into regions

syn_samples_per_region = 1000

def get_dataset_for_region(dataset, region_index, test_size_per_region=20):
    """Min-max scale and return data for a single, given region. The scaler must be fitted before.

    :param dataset: The dataset to get the regional data from
    :type dataset: pandas.DataFrame
    :param region_index: The index number of the region to return
    :type region_index: int
    :param test_size_per_region: The amount of values to separate for testing, default are 20
    :type test_size_per_region: int, optional
    :return: The dataset specific for the defined region, the test values, the test labels
    :rtype: tensorflow.python.data.ops.dataset_ops.PrefetchDataset, tuple of pandas.core.series.Series
    """
    region_ds = dataset[dataset['region'] == region_index]
    region_ds = region_ds.drop(columns=['region'])
    len = region_ds.shape[0]

    # The scaling into [0, 1] is not necessary anymore, it happens when the data loads already
    # region_ds[['age', 'bmi', 'children']] = scaler.transform(region_ds[['age', 'bmi', 'children']])

    X_test = region_ds.head(test_size_per_region)
    y_test = X_test.pop('charges')

    X_train = region_ds.tail(len - test_size_per_region)
    y_train = X_train.pop('charges')

    fed_train_dataset = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)))

    return (
        fed_train_dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER),
        (X_test, y_test)
    )


In [5]:
# Create test and train sets and put them into random_client_ds, use four clients which are independent of the region
def get_dataset_random_region(dataset, num_clients=4, test_size_per_region=20):
    """Creates a list with client datasets independent of the region.

    :param dataset: The dataset to get the regional data from
    :type dataset: pandas.DataFrame
    :param num_clients: the number of clients create (equal big datasets per client), default value is 4 clients
    :type num_clients: int, optional
    :param test_size_per_region: The amount of values to separate for testing, default are 20
    :type test_size_per_region: int, optional
    :return: List of the prepared dataset with one entry per region, the test values and labels for each region
    :rtype: List of (tensorflow.python.data.ops.dataset_ops.PrefetchDataset, tuple of pandas.core.series.Series)"""
    size_of_client_ds = int(dataset.shape[0] / num_clients)

    dataset_to_split = dataset.copy()
    dataset_to_split.pop("region")
    random_client_ds = []
    for i in range(num_clients):
        sampled = dataset_to_split.sample(n=size_of_client_ds)
        dataset_to_split.drop(sampled.index)

        X_test = sampled.head(test_size_per_region)
        y_test = X_test.pop('charges')

        X_train = sampled.tail(size_of_client_ds - test_size_per_region)
        y_train = X_train.pop('charges')

        fed_train_dataset = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)))

        train_set = fed_train_dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER)
        test_set = (X_test, y_test)

        random_client_ds.append((train_set, test_set))

    return random_client_ds

In [6]:
def create_keras_model(input_features=9):
    """Create neural network for given number of input features)

    :param input_features: The dimension of the first layer with represents the amount of features used
    :type input_features: int, optional
    :return: Created but not compiled model
    :rtype: keras.Model
    """
    return tf.keras.models.Sequential([
        # without region: tf.keras.layers.InputLayer(input_shape=(5,)),
        tf.keras.layers.InputLayer(input_shape=(input_features,)),
        tf.keras.layers.Dense(16, kernel_initializer='zeros'),
        tf.keras.layers.Dense(6, kernel_initializer='zeros'),
        tf.keras.layers.Dense(1, kernel_initializer='zeros'),
    ])


# A helper function for federated learning
def model_fn(input_features):
    """A function for TFF to create a local model during federated learning and return it as the correct type.
    This model uses 9 input features i.e. the regions are part of the features.

    :return: The LSTM model to be used as federated models
    :rtype: tff.learning.Model
    """
    # We _must_ create a new model here, and _not_ capture it from an external
    # scope. TFF will call this within different graph contexts.
    keras_model = create_keras_model(input_features)
    return tff.learning.from_keras_model(
        keras_model,
        # without region: input_spec=federated_insurance_data[0][0].element_spec,
        input_spec=random_client_ds[0][0].element_spec,
        loss=tf.keras.losses.MeanAbsoluteError(),
        metrics=[tf.keras.metrics.MeanAbsoluteError(),
                 # tfa.metrics.RSquare()
                 ]
    )

# Helper functions for different features as input
def model_fn_5():
    return model_fn(5)

def model_fn_9():
    return model_fn(9)

In [7]:
# Training data for clients with regional data (each client one region)
test_size_per_region = 20
regions = ['region0', 'region1', 'region2', 'region3']
federated_insurance_data = [get_dataset_for_region(dataset.drop(regions, axis=1), i, test_size_per_region=test_size_per_region)
                            for i in range(NUM_CLIENTS-1)]

2023-01-26 17:23:17.442527: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-26 17:23:17.442568: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-26 17:23:17.442593: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (svsram): /proc/driver/nvidia/version does not exist


In [8]:
# Training data for clients with regional independent data
random_client_ds = get_dataset_random_region(dataset, num_clients=NUM_CLIENTS, test_size_per_region=test_size_per_region)

### Training with one client per region

In [9]:
# Create iterative learning process which will perform the federated learning
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn_9,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.8),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=3))

In [10]:
# Create iterative learning process which will perform the federated learning
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn_5,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.8),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=3))

ValueError: in user code:

    File "/informatik1/students/home/0zadim/anaconda3/lib/python3.9/site-packages/tensorflow_federated/python/learning/templates/model_delta_client_work.py", line 176, in reduce_fn  *
        output = model.forward_pass(batch, training=True)
    File "/informatik1/students/home/0zadim/anaconda3/lib/python3.9/site-packages/tensorflow_federated/python/learning/framework/dataset_reduce.py", line 34, in _dataset_reduce_fn  *
        return dataset.reduce(initial_state=initial_state_fn(), reduce_func=reduce_fn)
    File "/informatik1/students/home/0zadim/anaconda3/lib/python3.9/site-packages/tensorflow_federated/python/learning/keras_utils.py", line 459, in forward_pass  *
        return self._forward_pass(batch_input, training=training)
    File "/informatik1/students/home/0zadim/anaconda3/lib/python3.9/site-packages/tensorflow_federated/python/learning/keras_utils.py", line 412, in _forward_pass  *
        predictions = self.predict_on_batch(inputs, training)
  

In [11]:
# The initial setup of the learning process
print(iterative_process.initialize.type_signature.formatted_representation())

( -> <
  global_model_weights=<
    trainable=<
      float32[9,16],
      float32[16],
      float32[16,6],
      float32[6],
      float32[6,1],
      float32[1]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64
  >
>@SERVER)


In [12]:
#summary_writer = tf.summary.create_file_writer(logdir+RUN_NAME)
state = iterative_process.initialize()

In [ ]:
# Train the federated model with random clients
for round_num in tqdm.tqdm(range(1, NUM_ROUNDS)):
    result = iterative_process.next(state, [f[0] for f in random_client_ds])
    state = result.state
    metrics = result.metrics
    for name, value in metrics['client_work']['train'].items():
        tf.summary.scalar(name, value, step=round_num)

 28%|██▊       | 41/149 [00:30<01:13,  1.47it/s]

In [ ]:
# Create the test data for model evaluation
X_test = pd.concat([f[1][0] for f in random_client_ds])
y_test = pd.concat([f[1][1] for f in random_client_ds])

test_sets = [tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(np.expand_dims(el[1][0], axis=0)), tf.convert_to_tensor(np.expand_dims(el[1][1], axis=0)))) for el in random_client_ds]

In [ ]:
# Model evaluation
evaluation = tff.learning.build_federated_evaluation(model_fn_9)
# print(evaluation.type_signature.formatted_representation())
model_weights = iterative_process.get_model_weights(state)
train_metrics = evaluation(model_weights, test_sets)
train_metrics


In [ ]:
# Create model from training results and evaluate
model = create_keras_model()
model_weights.assign_weights_to(model)
model.compile(
    loss=tf.losses.mae,
    optimizer=tf.optimizers.Adam(),
    metrics=["mae", 'mean_squared_error', RSquare()]
)
# The evaluation results, for technical reasons the metrics_names is called afterwards. However, its order fits to the results
print(model.evaluate(X_test, y_test))
print(model.metrics_names)


## Random clients (not ordered by region, as a test scenario):

In [26]:
random_client_no_regions = get_dataset_random_region(dataset.drop(regions, axis=1), num_clients=NUM_CLIENTS,
                                                     test_size_per_region=test_size_per_region)

In [27]:
random_client_ds

[(<PrefetchDataset element_spec=(TensorSpec(shape=(None, 9), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
  (           age  sex       bmi  children  smoker  region0  region1  region2  \
   578   0.739130    1  0.383105       0.2       0        0        0        0   
   610   0.630435    0  0.360775       0.2       0        0        0        1   
   569   0.652174    1  0.661959       0.4       1        0        1        0   
   1034  0.934783    1  0.603175       0.0       0        0        1        0   
   198   0.717391    0  0.056228       0.0       0        0        1        0   
   981   0.347826    1  0.145682       0.0       0        1        0        0   
   31    0.000000    0  0.278585       0.0       0        1        0        0   
   1256  0.717391    0  0.549502       0.6       0        0        1        0   
   1219  0.434783    0  0.383374       0.6       0        0        1        0   
   1320  0.282609    1  0.406376       0.

In [17]:
test_size_per_client = 20
size_of_client_ds = int(dataset.shape[0] / 4)

dataset_to_split = dataset.copy()
random_client_ds = []
for i in range(4):
    sampled = dataset_to_split.sample(n=size_of_client_ds)
    dataset_to_split.drop(sampled.index)

    X_test = sampled.head(test_size_per_region)
    y_test = X_test.pop('charges')

    X_train = sampled.tail(size_of_client_ds - test_size_per_region)
    #X_train = pd.concat([X_train, syn_region_ds], sort=False)
    y_train = X_train.pop('charges')

    fed_train_dataset = tf.data.Dataset.from_tensor_slices(
        (tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train)))

    train_set = fed_train_dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(BATCH_SIZE).prefetch(
        PREFETCH_BUFFER)
    test_set = (X_test, y_test)

    random_client_ds.append((train_set, test_set))




In [28]:
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn_5,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.8),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=3))


ValueError: in user code:

    File "/informatik1/students/home/0zadim/anaconda3/lib/python3.9/site-packages/tensorflow_federated/python/learning/templates/model_delta_client_work.py", line 176, in reduce_fn  *
        output = model.forward_pass(batch, training=True)
    File "/informatik1/students/home/0zadim/anaconda3/lib/python3.9/site-packages/tensorflow_federated/python/learning/framework/dataset_reduce.py", line 34, in _dataset_reduce_fn  *
        return dataset.reduce(initial_state=initial_state_fn(), reduce_func=reduce_fn)
    File "/informatik1/students/home/0zadim/anaconda3/lib/python3.9/site-packages/tensorflow_federated/python/learning/keras_utils.py", line 459, in forward_pass  *
        return self._forward_pass(batch_input, training=training)
    File "/informatik1/students/home/0zadim/anaconda3/lib/python3.9/site-packages/tensorflow_federated/python/learning/keras_utils.py", line 412, in _forward_pass  *
        predictions = self.predict_on_batch(inputs, training)
  

In [19]:
RUN_NAME = '0,8-3(150)-5-epochs-10-batch-WithRegion/'


In [20]:
print(iterative_process.initialize.type_signature.formatted_representation())
NUM_ROUNDS = 150
#@test {"skip": true}


( -> <
  global_model_weights=<
    trainable=<
      float32[9,16],
      float32[16],
      float32[16,6],
      float32[6],
      float32[6,1],
      float32[1]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=<>,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64
  >
>@SERVER)


In [21]:
#logdir = "/tmp/logs/scalars/training/"
#summary_writer = tf.summary.create_file_writer(logdir + RUN_NAME)
state = iterative_process.initialize()



[libprotobuf INFO google/protobuf/util/message_differencer.cc:1419] Proto type 'tensorflow.GraphDef' not found
[libprotobuf INFO google/protobuf/util/message_differencer.cc:1419] Proto type 'tensorflow.GraphDef' not found


In [22]:
#@test {"skip": true}
#with summary_writer.as_default():
for round_num in range(1, NUM_ROUNDS):
    result = iterative_process.next(state, [f[0] for f in random_client_ds])  #  [f[0] for f in federated_insurance_data])
    state = result.state
    metrics = result.metrics
    for name, value in metrics['client_work']['train'].items():
        tf.summary.scalar(name, value, step=round_num)
#@test {"skip": true}


In [23]:
X_test = pd.concat([f[1][0] for f in random_client_ds])
y_test = pd.concat([f[1][1] for f in random_client_ds])
#test_data = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(X_test), tf.convert_to_tensor(y_test)))

test_sets = [tf.data.Dataset.from_tensor_slices(
    (tf.convert_to_tensor(np.expand_dims(el[1][0], axis=0)), tf.convert_to_tensor(np.expand_dims(el[1][1], axis=0))))
    for el in random_client_ds]


In [24]:
evaluation = tff.learning.build_federated_evaluation(model_fn)
#print(evaluation.type_signature.formatted_representation())
model_weights = iterative_process.get_model_weights(state)
train_metrics = evaluation(model_weights, test_sets)
str(train_metrics)

[libprotobuf INFO google/protobuf/util/message_differencer.cc:1419] Proto type 'tensorflow.GraphDef' not found


"OrderedDict([('eval', OrderedDict([('mean_absolute_error', 8133.467), ('loss', 8133.4673), ('num_examples', 80), ('num_batches', 4)]))])"

In [33]:

model = create_keras_model()
model_weights.assign_weights_to(model)
model.compile(
    loss=tf.losses.mae,
    optimizer=tf.optimizers.Adam(),
    metrics=["mae", 'mean_squared_error', RSquare()]
)
model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 3ms/step - loss: 8413.9219 - mae: 8413.9219 - mean_squared_error: 170050112.0000 - r_square: -0.1058


[8413.921875, 8413.921875, 170050112.0, -0.1058434247970581]